# Black Hole Census: Qwen3-4B-Base

**Hypothesis test:** If black holes form at initialization and diffuse during post-training, then:
- Qwen3-4B-Base (pretrained only) should have C_b = 1 black hole with population P_b ≥ 2,211
- Qwen3-4B-Instruct-2507 (post-trained) has C_i = 13 black holes with population P_i ≈ 2,211

This notebook counts black holes in the Base model to test the prediction.

## Mathematical Background

A **black hole** is a set of tokens that share identical embedding vectors in γ-space:

$$\text{BH}_i = \{t_j : \gamma_{t_j} = v_i\}$$

where $|\text{BH}_i| \geq 2$ (degenerate tokens).

**Black hole count** C = number of distinct shared vectors  
**Black hole population** P = total number of tokens in all black holes

## Parameters

In [25]:
MODEL_NAME = "Qwen/Qwen3-4B-Base"
DEVICE = "mps"  # Apple Silicon

## Imports

In [26]:
import torch
from transformers import AutoModelForCausalLM
from collections import Counter

## Load Model

In [27]:
print(f"Loading {MODEL_NAME}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map=DEVICE
)
print("Model loaded successfully.")

Loading Qwen/Qwen3-4B-Base...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully.


## Extract Unembedding Matrix

In [28]:
gamma = model.lm_head.weight.data.cpu()  # [vocab_size, hidden_dim]
vocab_size, hidden_dim = gamma.shape

print(f"Vocabulary size: {vocab_size:,}")
print(f"Hidden dimension: {hidden_dim:,}")
print(f"Dtype: {gamma.dtype}")

Vocabulary size: 151,936
Hidden dimension: 2,560
Dtype: torch.bfloat16


## Find Black Holes

In [29]:
# Find unique vectors and their inverse mapping
print("Finding unique vectors...")
unique_vectors, inverse_indices = torch.unique(gamma, dim=0, return_inverse=True)

print(f"Total tokens: {vocab_size:,}")
print(f"Unique vectors: {len(unique_vectors):,}")
print(f"Duplicate vectors eliminated: {vocab_size - len(unique_vectors):,}")

Finding unique vectors...
Total tokens: 151,936
Unique vectors: 149,806
Duplicate vectors eliminated: 2,130


## Count Black Hole Populations

In [30]:
# Count how many tokens map to each unique vector
vector_populations = Counter(inverse_indices.tolist())

# Black holes are vectors with population ≥ 2
black_holes = {vec_id: pop for vec_id, pop in vector_populations.items() if pop >= 2}

C_b = len(black_holes)  # Black hole count
P_b = sum(black_holes.values())  # Total black hole population

print(f"\n{'='*60}")
print(f"BLACK HOLE CENSUS: Qwen3-4B-Base")
print(f"{'='*60}")
print(f"Black hole count (C_b): {C_b}")
print(f"Total black hole population (P_b): {P_b:,}")
print(f"{'='*60}\n")


BLACK HOLE CENSUS: Qwen3-4B-Base
Black hole count (C_b): 62
Total black hole population (P_b): 2,192



## Black Hole Details

In [31]:
# Sort black holes by population (largest first)
sorted_bhs = sorted(black_holes.items(), key=lambda x: x[1], reverse=True)

print("Black hole populations (sorted by size):\n")
for i, (vec_id, pop) in enumerate(sorted_bhs, 1):
    print(f"BH #{i}: {pop:,} tokens")

if C_b > 0:
    print(f"\nLargest black hole: {sorted_bhs[0][1]:,} tokens")
    if C_b > 1:
        print(f"Smallest black hole: {sorted_bhs[-1][1]:,} tokens")

Black hole populations (sorted by size):

BH #1: 582 tokens
BH #2: 456 tokens
BH #3: 296 tokens
BH #4: 159 tokens
BH #5: 99 tokens
BH #6: 80 tokens
BH #7: 56 tokens
BH #8: 52 tokens
BH #9: 45 tokens
BH #10: 36 tokens
BH #11: 30 tokens
BH #12: 29 tokens
BH #13: 26 tokens
BH #14: 24 tokens
BH #15: 19 tokens
BH #16: 16 tokens
BH #17: 15 tokens
BH #18: 10 tokens
BH #19: 8 tokens
BH #20: 8 tokens
BH #21: 8 tokens
BH #22: 7 tokens
BH #23: 7 tokens
BH #24: 7 tokens
BH #25: 6 tokens
BH #26: 6 tokens
BH #27: 6 tokens
BH #28: 5 tokens
BH #29: 5 tokens
BH #30: 5 tokens
BH #31: 5 tokens
BH #32: 4 tokens
BH #33: 4 tokens
BH #34: 4 tokens
BH #35: 3 tokens
BH #36: 3 tokens
BH #37: 3 tokens
BH #38: 3 tokens
BH #39: 3 tokens
BH #40: 3 tokens
BH #41: 3 tokens
BH #42: 3 tokens
BH #43: 3 tokens
BH #44: 3 tokens
BH #45: 3 tokens
BH #46: 2 tokens
BH #47: 2 tokens
BH #48: 2 tokens
BH #49: 2 tokens
BH #50: 2 tokens
BH #51: 2 tokens
BH #52: 2 tokens
BH #53: 2 tokens
BH #54: 2 tokens
BH #55: 2 tokens
BH #56: 2 

## Hypothesis Test Result

In [32]:
P_i = 2211  # Known population from Qwen3-4B-Instruct-2507
C_i = 13    # Known count from Qwen3-4B-Instruct-2507

print("\nHypothesis: Black holes form at initialization, diffuse during post-training\n")
print("Predictions:")
print(f"  Base model should have C_b = 1 (single primordial cluster)")
print(f"  Base model should have P_b ≥ {P_i:,} (all unused tokens together)\n")

print("Observations:")
print(f"  C_b = {C_b}")
print(f"  P_b = {P_b:,}\n")

print("Result:")
if C_b == 1 and P_b >= P_i:
    print("  ✓ HYPOTHESIS SUPPORTED")
    print("  Base model has single black hole with population ≥ Instruct total.")
elif C_b == 1 and P_b < P_i:
    print("  ✗ PARTIAL SUPPORT")
    print("  Single black hole as predicted, but population smaller than expected.")
    print("  Some tokens may have been rescued during pretraining.")
elif C_b > 1:
    print("  ✗ HYPOTHESIS CHALLENGED")
    print("  Multiple black holes found in base model.")
    print("  Diffusion may occur during pretraining, not just post-training.")
else:
    print("  ✗ UNEXPECTED RESULT")
    print("  No black holes found in base model.")


Hypothesis: Black holes form at initialization, diffuse during post-training

Predictions:
  Base model should have C_b = 1 (single primordial cluster)
  Base model should have P_b ≥ 2,211 (all unused tokens together)

Observations:
  C_b = 62
  P_b = 2,192

Result:
  ✗ HYPOTHESIS CHALLENGED
  Multiple black holes found in base model.
  Diffusion may occur during pretraining, not just post-training.
